In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from geopy.distance import geodesic

from sklearn import linear_model, model_selection

In [ ]:
# load train and test data

# train set
df_mess_train = pd.read_csv('mess_train_list.csv')

# test set
df_mess_test = pd.read_csv('mess_test_list.csv')

# position associated to train set
pos_train = pd.read_csv('pos_train_list.csv') 

In [ ]:
df_mess_train.head()

In [ ]:
df_mess_train.describe()

In [ ]:
pos_train.head()

In [ ]:
pos_train.describe()

In [ ]:
# determine all Base stations that received at least 1 message
listOfBs = np.union1d(np.unique(df_mess_train['bsid']),
                      np.unique(df_mess_test['bsid'])) 

print(f"Number of stations: {len(listOfBs)}")

In [ ]:
df_mess_train['did'].unique().shape

In [ ]:
# Feature Matrix construction 
def feat_mat_const(df_mess_train, listOfBs):
    df = df_mess_train
    df['active'] = 1
    #df = df[df['bs_lat'] < 50]
    df = df.pivot_table(index='messid', 
                                    values=['active', 'nseq', 'rssi', 'bs_lat', 'bs_lng'], 
                                    columns=['bsid'],
                                    fill_value=0)
    return df.reorder_levels([1, 0], axis=1).sort_index(level=0, axis=1)

In [ ]:
# ground truth construction
def ground_truth_const(df_mess_train, pos_train):
    
    df = pd.concat([df_mess_train[['messid', 'bs_lat']], pos_train], axis=1)
    df_mean = df.groupby('messid').mean()

    return df_mean['lat'], df_mean['lng']

In [ ]:
df_feat = feat_mat_const(df_mess_train, listOfBs)
df_feat.head()

In [ ]:
df_feat.shape

In [ ]:
ground_truth_lat, ground_truth_lng = ground_truth_const(df_mess_train, pos_train)
ground_truth_lat.shape

## Make regressor and prediction using the train set

In [ ]:
def regressor_and_predict(df_feat, ground_truth_lat, ground_truth_lng, df_test):
    
    # train regressor and make prediction in the train set
    # Input: df_feat: feature matrix used to train regressor
    #        ground_truth_lat: df_feat associated latitude
    #        ground_truth_lng: df_feat associated longitude
    #        df_test: data frame used for prediction
    # Output: y_pred_lat, y_pred_lng

    X_train = np.array(df_feat);
    reg = linear_model.LinearRegression()

    reg.fit(X_train, ground_truth_lat);
    y_pred_lat = reg.predict(df_test) 

    reg.fit(X_train, ground_truth_lng);
    y_pred_lng = reg.predict(df_test) 
    
    return y_pred_lat, y_pred_lng

In [ ]:
y_pred_lat, y_pred_lng = regressor_and_predict(df_feat, ground_truth_lat, ground_truth_lng, df_feat)

## Cross validation

In [ ]:
reg = linear_model.LinearRegression()
y_pred_lng = model_selection.cross_val_predict(reg, df_feat, ground_truth_lng, cv=5)
y_pred_lat = model_selection.cross_val_predict(reg, df_feat, ground_truth_lat, cv=5)

## Evaluate result

In [ ]:
def vincenty_vec(vec_coord):
    """ Now using geodesic distance instead of Vincenty """
    vin_vec_dist = np.zeros(vec_coord.shape[0])
    if vec_coord.shape[1] != 4:
        print('ERROR: Bad number of columns (shall be = 4)')
    else:
        vin_vec_dist = [geodesic(vec_coord[m, 0:2], vec_coord[m, 2:]).meters for m in range(vec_coord.shape[0])]

    return vin_vec_dist

In [ ]:
# evaluate distance error for each predicted point
def eval_geoloc(y_train_lat , y_train_lng, y_pred_lat, y_pred_lng):
    vec_coord = np.array([y_train_lat , y_train_lng, y_pred_lat, y_pred_lng])
    err_vec = vincenty_vec(np.transpose(vec_coord))
    
    return err_vec

Remove asburd values of latitude and longitude

In [ ]:
correct = (y_pred_lat > -90) & (y_pred_lat < 90) \
        & (y_pred_lng > -180) & (y_pred_lng <= 180)

y_pred_lat_clean = y_pred_lat[correct]
y_pred_lng_clean = y_pred_lng[correct]

ground_truth_lat_clean = ground_truth_lat[correct]
ground_truth_lng_clean = ground_truth_lng[correct]

In [ ]:
err_vec = eval_geoloc(ground_truth_lat_clean, ground_truth_lng_clean, y_pred_lat_clean, y_pred_lng_clean)

## Plot error distribution

In [ ]:
values, base = np.histogram(err_vec, bins=50000)
cumulative = np.cumsum(values) 

plt.figure()

plt.grid()

plt.plot(base[:-1]/1000, cumulative / np.float(np.sum(values))  * 100.0,
         label="Opt LLR", c='blue')

# plot error @ 80%
plt.axvline(x=np.percentile(err_vec, 80)/1000., ymin=0, ymax=100,
            linestyle='dashed', color='red')

plt.xlabel('Distance Error (km)')
plt.ylabel('Cum proba (%)')
plt.axis([0, 30, 0, 100]) 

plt.title('Error Cumulative Probability')
plt.legend()

plt.show()

## Error criterion

In [ ]:
print(f"error @ 80% = {np.percentile(err_vec, 80):.1f} m")

## Construct test prediction

In [ ]:
df_mess_test.head()

In [ ]:
df_feat_test = feat_mat_const(df_mess_test, listOfBs)

In [ ]:
y_pred_lat, y_pred_lng = regressor_and_predict(df_feat, ground_truth_lat, ground_truth_lng, df_feat_test)

In [ ]:
test_res = pd.DataFrame(np.array([y_pred_lat, y_pred_lng]).T, columns = ['lat', 'lng'])
test_res['messid'] = id_list_test

In [ ]:
test_res.to_csv('pred_pos_test_list.csv', index=False)

In [ ]:
test_res.head()